In [1]:
import numpy as np
import os
from codeStore import support_fun as spf
import importlib


In [2]:
# case 1, ecoli (head Force) model in finite pipe. 
ellipse_centerx_list = [0.75, 0.5, 0]
rot_theta_list = np.linspace(0, 2, 11) # true theta=rot_theta_list*np.pi
rs1_list = (0.1, 0.2, 0.3)
PoiseuilleStrength_list = np.linspace(-0.5, 0.5, 11)
# PoiseuilleStrength_list = (0, )
job_dir = 'case1a'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)
    
t_name = os.path.join(job_dir, 'run.sh')
n_pbs = 0
with open(t_name, 'w') as frun:
    # create .pbs file
    for ellipse_centerx_i in ellipse_centerx_list:
        for rot_theta_i in rot_theta_list:
            for rs1_i in rs1_list:
                if rs1_i + ellipse_centerx_i < 1:
                    job_name = 'headF_x%.2f_rot%.2f_rs1%.2f' % (ellipse_centerx_i, rot_theta_i, rs1_i)
                    t_name = os.path.join(job_dir, '%s.pbs' % job_name)
                    with open(t_name, 'w') as fpbs:
                        spf.write_pbs_head(fpbs, job_name)
                        for PoiseuilleStrength_i in PoiseuilleStrength_list:
                            output_name = '%s_P%.2f' % (job_name, PoiseuilleStrength_i)
                            fpbs.write('mpirun -n 24 python ')
                            fpbs.write(' ../head_Force_in_pipe.py ')
                            fpbs.write(' -f %s ' % output_name)
                            fpbs.write(' -sm %s ' % 'lg_rs')
                            fpbs.write(' -legendre_m %d ' % 3)
                            fpbs.write(' -legendre_k %d ' % 2)
                            fpbs.write(' -epsilon %f ' % 3)
                            fpbs.write(' -PoiseuilleStrength %f ' % PoiseuilleStrength_i)
                            fpbs.write(' -ecoli_tail_strength %f ' % 1)
                            fpbs.write(' -rs1 %f ' % rs1_i)
                            fpbs.write(' -rs2 %f ' % 0.1)
                            fpbs.write(' -ds %f ' % 0.01)
                            fpbs.write(' -ellipse_centerx %f ' % ellipse_centerx_i)
                            fpbs.write(' -ellipse_centery %f ' % 0)
                            fpbs.write(' -ellipse_centerz %f ' % 0)
                            fpbs.write(' -rot_theta %f ' % rot_theta_i)
                            fpbs.write(' -finite_pipe_length %f ' % 10)
                            fpbs.write(' -finite_pipe_cover %d ' % 1)
                            fpbs.write(' -finite_pipe_ntheta %d ' % 100)
                            fpbs.write(' -vtk_matname %s ' % 'pipe_20181209')
                            fpbs.write(' -ksp_max_it %d ' % 500)
                            fpbs.write(' > %s.txt \n\n' % output_name)
                    # write to .sh file
                    frun.write('qsub %s.pbs\n' % job_name)
                    n_pbs = n_pbs + 1
    
    frun.write('\n')
    print('n_pbs = ', n_pbs)

n_pbs =  88


In [3]:
# case 2, ecoli (head Force) model in finite pipe. motion loop
ellipse_centerx_list = (0.75, 0.5, 0)
rot_theta_list = (0, 1/3, ) # true theta=rot_theta_list*np.pi
rs1_list = (0.1, )
PoiseuilleStrength_list = (0, 0.05)
# PoiseuilleStrength_list = (0, )
PWD = os.getcwd()
job_dir = 'case2a'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)
    
t_name = os.path.join(job_dir, 'run.sh')
n_pbs = 0
with open(t_name, 'w') as frun:
    # create .pbs file
    frun.write('t_dir=$PWD \n')
    for ellipse_centerx_i in ellipse_centerx_list:
        for rs1_i in rs1_list:
            if rs1_i + ellipse_centerx_i < 1:
                for rot_theta_i in rot_theta_list:
                    for PoiseuilleStrength_i in PoiseuilleStrength_list:
                        job_name = 'headF_x%.2f_rot%.2f_rs1%.2f_P%.2f' % \
                                   (ellipse_centerx_i, rot_theta_i, rs1_i, PoiseuilleStrength_i)
                        print(job_name)
                        t_path = os.path.join(job_dir, job_name)
                        if not os.path.exists(t_path):
                            os.makedirs(t_path)
                        t_name = os.path.join(t_path, '%s.pbs' % job_name)
                        with open(t_name, 'w') as fpbs:
                            spf.write_pbs_head(fpbs, job_name)                           
                            fpbs.write('mpirun -n 24 python ')
                            fpbs.write(' ../../motion_head_Force_pipe.py ')
                            fpbs.write(' -f %s ' % job_name)
                            fpbs.write(' -sm %s ' % 'lg_rs')
                            fpbs.write(' -legendre_m %d ' % 3)
                            fpbs.write(' -legendre_k %d ' % 2)
                            fpbs.write(' -epsilon %f ' % 3)
                            fpbs.write(' -PoiseuilleStrength %f ' % PoiseuilleStrength_i)
                            fpbs.write(' -ecoli_tail_strength %f ' % 1)
                            fpbs.write(' -rs1 %f ' % rs1_i)
                            fpbs.write(' -rs2 %f ' % 0.1)
                            fpbs.write(' -ds %f ' % 0.01)
                            fpbs.write(' -ellipse_centerx %f ' % ellipse_centerx_i)
                            fpbs.write(' -ellipse_centery %f ' % 0)
                            fpbs.write(' -ellipse_centerz %f ' % 0)
                            fpbs.write(' -rot_theta %f ' % rot_theta_i)
                            fpbs.write(' -finite_pipe_length %f ' % 10)
                            fpbs.write(' -finite_pipe_cover %d ' % 1)
                            fpbs.write(' -finite_pipe_ntheta %d ' % 100)
                            fpbs.write(' -vtk_matname %s ' % 'pipe_20181209')
                            fpbs.write(' -ksp_max_it %d ' % 500)
                            fpbs.write(' -max_iter %d ' % 500)
                            fpbs.write(' -eval_dt %f ' % 0.01)
                            fpbs.write(' > %s.txt \n\n' % job_name)
                        # write to .sh file
                        frun.write('cd $t_dir/%s\n' % job_name)
                        frun.write('qsub %s.pbs\n\n' % job_name)
                        n_pbs = n_pbs + 1
    
    frun.write('\n')
    print('n_pbs = ', n_pbs)

headF_x0.75_rot0.00_rs10.10_P0.00
headF_x0.75_rot0.00_rs10.10_P0.05
headF_x0.75_rot0.33_rs10.10_P0.00
headF_x0.75_rot0.33_rs10.10_P0.05
headF_x0.50_rot0.00_rs10.10_P0.00
headF_x0.50_rot0.00_rs10.10_P0.05
headF_x0.50_rot0.33_rs10.10_P0.00
headF_x0.50_rot0.33_rs10.10_P0.05
headF_x0.00_rot0.00_rs10.10_P0.00
headF_x0.00_rot0.00_rs10.10_P0.05
headF_x0.00_rot0.33_rs10.10_P0.00
headF_x0.00_rot0.33_rs10.10_P0.05
n_pbs =  12


In [3]:
# ecoli_shear1, ecoli model in infinite shear flow. motion ecoli 

# eval_dt_list = [0.1, 0.05, 0.01,]
# update_order_list = [1, 2, 3, 4, 5]
# # eval_dt_fct_list = []
# max_iter = 500
# job_dir = 'ecoli_shear1c'
# write_pbs_head = spf.write_pbs_head

# eval_dt_list = [1, 10, 100]
# update_order_list = [1, 2, 3, 4, 5]
# # eval_dt_fct_list = []
# max_iter = 500
# job_dir = 'ecoli_shear1d'
# write_pbs_head = spf.write_pbs_head

# eval_dt_list = [0.5, 2, 4, 8]
# update_order_list = [1, 2, 3, 4, 5]
# # eval_dt_fct_list = []
# max_iter = 500
# job_dir = 'ecoli_shear1e'
# write_pbs_head = spf.write_pbs_head

eval_dt_list = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]
update_order_list = [1, 2, 3, 4, 5]
# eval_dt_fct_list = []
max_iter = 500
job_dir = 'ecoli_shear1f'
write_pbs_head = spf.write_pbs_head

PWD = os.getcwd()
if not os.path.exists(job_dir):
    os.makedirs(job_dir)
    
t_name = os.path.join(job_dir, 'run.sh')
n_pbs = 0
with open(t_name, 'w') as frun:
    # create .pbs file
    frun.write('t_dir=$PWD \n')
    for eval_dt_i in eval_dt_list:
        for update_order_i in update_order_list:
            job_name = 'eq_dt%0.3f_O%1d' % (eval_dt_i, update_order_i)
            print(job_name)
            t_path = os.path.join(job_dir, job_name)
            if not os.path.exists(t_path):
                os.makedirs(t_path)
            t_name = os.path.join(t_path, '%s.pbs' % job_name)
            with open(t_name, 'w') as fpbs:
                write_pbs_head(fpbs, job_name)                           
                fpbs.write('mpirun -n 24 python ')
                fpbs.write(' ../../motion_ecoli.py ')
                fpbs.write(' -f %s ' % job_name)
                fpbs.write(' -sm %s ' % 'pf')
                fpbs.write(' -pickProblem %d ' % 0)
                fpbs.write(' -save_singleEcoli_vtk %d ' % 0)
                fpbs.write(' -rh1 %f ' % 0.1)
                fpbs.write(' -rh2 %f ' % 0.03)
                fpbs.write(' -ch %f ' % 3)
                fpbs.write(' -nth %d ' % 20)
                fpbs.write(' -eh %f ' % -1)
                fpbs.write(' -ph %f ' % (2/3))
                fpbs.write(' -hfct %f ' % 1)
                fpbs.write(' -n_tail %d ' % 1)
                fpbs.write(' -with_cover %d ' % 2)
                fpbs.write(' -left_hand %d ' % 0)
                fpbs.write(' -rs1 %f ' % 1.5)
                fpbs.write(' -rs2 %f ' % 0.5)
                fpbs.write(' -ds %f ' % 0.07)
                fpbs.write(' -es %f ' % -1)
                fpbs.write(' -with_T_geo %d ' % 0)
                fpbs.write(' -dist_hs %f ' % 0.5)
                fpbs.write(' -ksp_max_it %d ' % 500)
                fpbs.write(' -plot_geo %d ' % 0)
                fpbs.write(' -rel_wsz %f ' % 0)
                fpbs.write(' -rel_whz %f ' % 1)
                fpbs.write(' -ffweight %f ' % 2)
                fpbs.write(' -zoom_factor %f ' % 1)
                fpbs.write(' -max_iter %d ' % max_iter)
                fpbs.write(' -update_order %d ' % update_order_i)
                fpbs.write(' -eval_dt %f ' % eval_dt_i)
                fpbs.write(' -planeShearRatex %f ' % 0.003)
                fpbs.write(' > %s.txt \n\n' % job_name)
            # write to .sh file
            frun.write('cd $t_dir/%s\n' % job_name)
            frun.write('qsub %s.pbs\n\n' % job_name)
            n_pbs = n_pbs + 1
    
    frun.write('\n')
    print('n_pbs = ', n_pbs)

eq_dt0.000_O1
eq_dt0.000_O2
eq_dt0.000_O3
eq_dt0.000_O4
eq_dt0.000_O5
eq_dt0.001_O1
eq_dt0.001_O2
eq_dt0.001_O3
eq_dt0.001_O4
eq_dt0.001_O5
eq_dt0.001_O1
eq_dt0.001_O2
eq_dt0.001_O3
eq_dt0.001_O4
eq_dt0.001_O5
eq_dt0.005_O1
eq_dt0.005_O2
eq_dt0.005_O3
eq_dt0.005_O4
eq_dt0.005_O5
eq_dt0.010_O1
eq_dt0.010_O2
eq_dt0.010_O3
eq_dt0.010_O4
eq_dt0.010_O5
eq_dt0.050_O1
eq_dt0.050_O2
eq_dt0.050_O3
eq_dt0.050_O4
eq_dt0.050_O5
n_pbs =  30


In [24]:
# head_shear, head-force model in infinite shear flow. motion ecoli 

# update_order_list = [1, ]
# max_iter = 10001
# update_fun = 'Adams_Bashforth_Methods'
# job_dir = 'head_shear1a'
# rs2_list =     np.array([0.1,   0.3,   0.5,   0.7,   0.9,   1,    3,    5,    7,    9])
# ds_list  =     np.array([0.012, 0.022, 0.030, 0.035, 0.042, 0.05, 0.10, 0.17, 0.23, 0.30])
# eval_dt_list = np.array([1,     0.15,   0.10,  0.03,  0.06, 0.06, 0.03, 0.03, 0.03, 0.03]) * 0.1
# z_list = np.linspace(-3, 3, 5)
# importlib.reload(spf)
# write_pbs_head = spf.write_pbs_head_newturb

# update_order_list = [1, ]
# max_iter = 10001
# update_fun = 'Adams_Bashforth_Methods'
# job_dir = 'head_shear1b'
# rs2_list =     np.array([3,    5,    7,    9])
# ds_list  =     np.array([0.10, 0.17, 0.23, 0.30])
# eval_dt_list = np.array([0.15, 0.24, 0.3 , 0.45]) * 0.1
# z_list = np.linspace(-3, 3, 5)
# importlib.reload(spf)
# write_pbs_head = spf.write_pbs_head_newturb

update_order_list = [1, 2, 3, 4, 5]
max_iter = 10001
update_fun = 'Adams_Bashforth_Methods'
job_dir = 'head_shear1c'
rs2_list =     np.array([0.1,   9])
ds_list  =     np.array([0.012, 0.30])
eval_dt_list = np.array([1,     0.45]) * 0.1
z_list = [3, ]
importlib.reload(spf)
write_pbs_head = spf.write_pbs_head_newturb

PWD = os.getcwd()
if not os.path.exists(job_dir):
    os.makedirs(job_dir)
    
t_name = os.path.join(job_dir, 'run.sh')
n_pbs = 0
with open(t_name, 'w') as frun:
    # create .pbs file
    frun.write('t_dir=$PWD \n')
    for update_order_i in update_order_list:
        for rs2_i, dsi, eval_dt_i in zip(rs2_list, ds_list, eval_dt_list):
            for zi in z_list:
                job_name = 'eq_a%05.2f_o%1d' % (rs2_i, update_order_i)
                t_path = job_dir
                t_name = os.path.join(t_path, '%s.pbs' % job_name)
                with open(t_name, 'w') as fpbs:
                    write_pbs_head(fpbs, job_name)                           
                    fpbs.write('mpirun -n 24 python ')
                    fpbs.write(' ../motion_head_Force.py ')
                    fpbs.write(' -f %s ' % job_name)
                    fpbs.write(' -sm %s ' % 'lg_rs')
                    fpbs.write(' -pickProblem %d ' % 0)
                    fpbs.write(' -save_singleEcoli_vtk %d ' % 0)
                    fpbs.write(' -rs1 %f ' % 1)
                    fpbs.write(' -rs2 %f ' % rs2_i)
                    fpbs.write(' -ds %f ' % dsi)
                    fpbs.write(' -es %f ' % -1)
                    fpbs.write(' -ellipse_centerz %f ' % zi)
                    fpbs.write(' -ksp_max_it %d ' % 500)
                    fpbs.write(' -plot_geo %d ' % 0)
                    fpbs.write(' -ffweight %f ' % 2)
                    fpbs.write(' -zoom_factor %f ' % 1)
                    fpbs.write(' -max_iter %d ' % max_iter)
                    fpbs.write(' -update_order %d ' % update_order_i)
                    fpbs.write(' -update_fun %s ' % update_fun)
                    fpbs.write(' -eval_dt %f ' % eval_dt_i)
                    fpbs.write(' -ecoli_tail_strength %f ' % 0.0)
                    fpbs.write(' -planeShearRatex %f ' % 1)
                    fpbs.write(' > %s.txt \n\n' % job_name)
                # write to .sh file
                frun.write('qsub %s.pbs\n\n' % job_name)
                n_pbs = n_pbs + 1
    
    frun.write('\n')
    print('n_pbs = ', n_pbs)

n_pbs =  10


In [36]:
# head_shear, head-velocity model in infinite shear flow. motion ecoli 
# two nondimentional parameters: /alpha=rs_1/rs_1, /kappa=|v|/(z*/tau)

update_order_list = [1, ]
max_iter = 3001
update_fun = 'Adams_Moulton_Methods'
job_dir = 'head_shear_velocity1a'
rs2_list =     np.array([0.1,   1,    9])
ds_list  =     np.array([0.012, 0.1,  0.3])
eval_dt_list = np.array([1,     0.06, 0.45]) * 0.1
z_list = np.linspace(-1, 1, 5)
planeShearRatex = 2
ecoli_velocity = 1
importlib.reload(spf)
write_pbs_head = spf.write_pbs_head_newturb

PWD = os.getcwd()
if not os.path.exists(job_dir):
    os.makedirs(job_dir)
    
t_name = os.path.join(job_dir, 'run.sh')
n_pbs = 0
with open(t_name, 'w') as frun:
    # create .pbs file
    frun.write('t_dir=$PWD \n')
    for update_order_i in update_order_list:
        for rs2_i, dsi, eval_dt_i in zip(rs2_list, ds_list, eval_dt_list):
            for zi in z_list:
                job_name = 'eq_a%05.2f_z%05.2f' % (rs2_i, zi)
                t_path = job_dir
                t_name = os.path.join(t_path, '%s.pbs' % job_name)
                with open(t_name, 'w') as fpbs:
                    write_pbs_head(fpbs, job_name)                           
                    fpbs.write('mpirun -n 24 python ')
                    fpbs.write(' ../motion_head_speed.py ')
                    fpbs.write(' -f %s ' % job_name)
                    fpbs.write(' -sm %s ' % 'lg_rs')
                    fpbs.write(' -pickProblem %d ' % 0)
                    fpbs.write(' -save_singleEcoli_vtk %d ' % 0)
                    fpbs.write(' -rs1 %f ' % 1)
                    fpbs.write(' -rs2 %f ' % rs2_i)
                    fpbs.write(' -ds %f ' % dsi)
                    fpbs.write(' -es %f ' % -1)
                    fpbs.write(' -ellipse_centerz %f ' % zi)
                    fpbs.write(' -ecoli_velocity %f ' % ecoli_velocity)
                    fpbs.write(' -planeShearRatex %f ' % planeShearRatex)
                    fpbs.write(' -ksp_max_it %d ' % 500)
                    fpbs.write(' -plot_geo %d ' % 0)
                    fpbs.write(' -ffweight %f ' % 2)
                    fpbs.write(' -zoom_factor %f ' % 1)
                    fpbs.write(' -max_iter %d ' % max_iter)
                    fpbs.write(' -update_order %d ' % update_order_i)
                    fpbs.write(' -update_fun %s ' % update_fun)
                    fpbs.write(' -eval_dt %f ' % eval_dt_i)
                    fpbs.write(' > %s.txt \n\n' % job_name)
                # write to .sh file
                frun.write('qsub %s.pbs\n\n' % job_name)
                n_pbs = n_pbs + 1
    
    frun.write('\n')
    print('n_pbs = ', n_pbs)

n_pbs =  15
